In [2]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
        
clearall()

In [3]:
############
#### 78
###########

In [4]:
############################
######## LIBRAIRIES ########
############################
print('Importing Librairies...')
from sklearn.linear_model import LinearRegression
import colorama as col
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import seaborn as sns
import time
import os
import datetime as dt
import numpy as np
import fxcmpy
import socketio
import shelve
import talib
import statistics
import pyttsx3

#from numpy import loadtxt
#from functools import reduce

number = 10000

_total = 0
_cash_ini = 200000
_cash = _cash_ini
_rate = 1
#_flag = 0

DATE = []
CONTRACT = []
OPEN_POZ = []
CLOSE_POZ = []
RATE_OPEN_POZ = []
RATE_CLOSE_POZ = []
PNL_LAT = []
PNL_REAL = []
TOTAL_OPEN = []
TOTAL_CLOSE = []
PRICE_BUY = []
PRICE_SELL = []
DER_POZ = []
TOTAL_PNL_LAT = []
TOTAL_PNL_REAL = []
_cash = _cash_ini
WINNERS = []
LOOSERS = []
SIGNAL = []
OPEN = []
OPEN_BID = []
OPEN_ASK = []



_year_bottom = '2020-01-01 00:00:01'
_year_top = '2020-12-31 23:59:59'

_verbose = 1

engine = pyttsx3.init()

print('Librairies imported\n')
print('version fxcmpy :',fxcmpy.__version__)
%matplotlib inline
plt.style.use('seaborn')
engine.say("librairie loaded")
engine.runAndWait()

df_resultats = pd.DataFrame(index=['TimeFrame 1','TimeFrame2','Equity','Nbre Winners','Nbre Loosers','Max lenght of trade','Min lenght of trade',\
    'Average lenght of trade','Cumul pnl'])

x = 'EUR/USD'

print('Prêt')

Importing Librairies...
Librairies imported

version fxcmpy : 1.2.6
Prêt


In [9]:
%%time

engine.say("קדימה")
engine.runAndWait()

print(col.Fore.MAGENTA,'Le rate du ticker',x,'est à ',_rate,col.Style.RESET_ALL)

print('Chargement des bases')

data = pd.read_csv('Base_Clean/'+x.replace('/','')+'m5.csv')
data2 = pd.read_csv('Base_Clean/'+x.replace('/','')+'W1.csv')

data.index = pd.to_datetime(data['Date'])
data = data.drop(['Date','Total'],axis=1)
data.rename(columns={'OpenBid':'Open_Bid','HighBid':'High_Bid','LowBid':'Low_Bid','CloseBid':'Close_Bid',\
    'OpenAsk':'Open_Ask','HighAsk':'High_Ask','LowAsk':'Low_Ask','CloseAsk':'Close_Ask'},inplace = True)

data2['Date'] = data2['Date'].shift(-1)
data2.index = pd.to_datetime(data2['Date'])
data2 = data2.drop(['Date','Total'],axis=1)
data2.rename(columns={'OpenBid':'Open_Bid','HighBid':'High_Bid','LowBid':'Low_Bid','CloseBid':'Close_Bid',\
    'OpenAsk':'Open_Ask','HighAsk':'High_Ask','LowAsk':'Low_Ask','CloseAsk':'Close_Ask'},inplace = True)

data = data[(data.index >= _year_bottom) & (data.index <= _year_top)]
data2 = data2[(data2.index >= _year_bottom) & (data2.index <= _year_top)]

price = data.copy()
price2 = data2.copy()

print('Bases chargées')
print('Préparation des dataframes')

if '/JPY' in x:
    _pip_factor = 100
else:
    _pip_factor = 10000
price2['Range_Bid'] = (price2.Open_Bid.shift(1) - price2.Close_Bid.shift(1)) * _pip_factor
price2['Range_Ask'] = (price2.Open_Ask.shift(1) - price2.Close_Ask.shift(1)) * _pip_factor
price2['Range'] = (price2.Open.shift(1) - price2.Close.shift(1)) * _pip_factor
price2['OpenBid_Shift'] = price2['Open_Bid'].shift(-1)
price2['OpenAsk_Shift'] = price2['Open_Ask'].shift(-1)
price2['Open_Shift'] = price2['Open'].shift(-1)
price2['Signal'] = np.where((price2.Range_Ask >= _atr) | (price2.Range_Bid >= _atr) | (price2.Range >= _atr),1,\
    np.where((price2.Range_Ask <= -_atr) | (price2.Range_Bid <= -_atr) | (price2.Range <= -_atr),-1,0))
price2['TimeRange']= [price2.index[i].strftime(format='%Y-%m-%d') for i in range(len(price2))]
price['TimeRange']= [price.index[i].strftime(format='%Y-%m-%d') for i in range(len(price))]

print('TETEL process effectué')

print(col.Fore.CYAN,'ENTERING THE BACKTEST',col.Style.RESET_ALL)

for i in range(len(price)):
    try:
        SIGNAL.append(price2[price2.TimeRange == price.TimeRange[i]].Signal.values[0])
        OPEN.append(price2[price2.TimeRange == price.TimeRange[i]].Open_Shift.values[0])
        OPEN_BID.append(price2[price2.TimeRange == price.TimeRange[i]].OpenBid_Shift.values[0])
        OPEN_ASK.append(price2[price2.TimeRange == price.TimeRange[i]].OpenAsk_Shift.values[0])
    except:
        SIGNAL.append(999)
        OPEN.append(999)
        OPEN_BID.append(999)
        OPEN_ASK.append(999)
price['Signal'] = SIGNAL
price['OpenBid_Shift'] = OPEN_BID
price['OpenAsk_Shift'] = OPEN_ASK
price['Open_Shift'] = OPEN

price = price[(price.Signal != 999) & (price.OpenBid_Shift != 999) & (price.OpenAsk_Shift != 999)]
price = price.dropna()

_position = 0
_equity = 0
_nbtransactions = 0
backtest_graph = pd.DataFrame()
EQUITY = [_cash]
CASH = [_cash]
_winner = 0
_looser = 0
_index_entry = 0
TRADE_DURATION = []
_average_duration = 0
_size = 5000

PRICE_BUY = []
PRICE_SELL = []

_total = 0

_target = 0.006
_sl = 0.0015
_spread_filter = 0.00050
_filtre_multipoz = 0.5 # En %
_filtre_multipoz = _filtre_multipoz/100

_open_buy = 0
_open_sell = 0
_minipoz = 0

for i in range(0,len(price)):
    
    if _position == 0 :
        # BUY SIGNAL
        if price.Signal[i] == -1 : # -1 and (price.OpenBid_Shift[i] - price.High_Bid[i]) * _pip_factor < -15 and \
            #(price.OpenBid_Shift[i] - price.High_Bid[i]) * _pip_factor > -35 :
            _pnl = 0
            _open_buy += 1
            _equity = _cash + _pnl
            EQUITY.append(_equity)
            CASH.append(_cash)
            _position = 1
            _index_entry = i
            _nbtransactions += 1
            price_buy = price.Close_Ask.iloc[i]
            PRICE_BUY.append(price_buy)
            _price_buy_mean = round(sum(PRICE_BUY)/len(PRICE_BUY),5)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print('Position 1 bought at', price_buy,'(verification liste',PRICE_BUY[-1],')')

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(1)
            CLOSE_POZ.append(0)
            RATE_OPEN_POZ.append(price_buy)
            RATE_CLOSE_POZ.append(0)
            PNL_LAT.append(_pnl)
            PNL_REAL.append(0)
            TOTAL_OPEN.append(1) 
            TOTAL_PNL_LAT.append(_pnl)
            TOTAL_PNL_REAL.append(0) 

        
        
        # SELL SIGNAL
        elif price.Signal[i] == 1 : # 1 and (price.OpenAsk_Shift[i] - price.Low_Ask[i]) * _pip_factor > 15 and\
            #(price.OpenAsk_Shift[i] - price.Low_Ask[i]) * _pip_factor < 35 :
            _pnl = 0
            _open_sell += 1
            _equity = _cash + _pnl
            EQUITY.append(_equity)
            CASH.append(_cash)
            _index_entry = i
            _position = -1
            _nbtransactions += 1
            price_sell = price.Close_Bid.iloc[i]
            PRICE_SELL.append(price_sell)
            _price_sell_mean = round(sum(PRICE_SELL)/len(PRICE_SELL),5)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print('Position 1 sold at', price_sell,'(verification liste',PRICE_SELL[-1],')')

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(-1)
            CLOSE_POZ.append(0)
            RATE_OPEN_POZ.append(price_sell)
            RATE_CLOSE_POZ.append(0) 
            PNL_LAT.append(_pnl)
            PNL_REAL.append(0)
            TOTAL_PNL_LAT.append(_pnl)
            TOTAL_PNL_REAL.append(0)

            TOTAL_OPEN.append(1)

        else :
            _pnl = 0
            _equity = _cash + _pnl
            EQUITY.append(_equity)
            CASH.append(_cash)
    
    elif _position == 1:

        if price.Signal[i] == -1 : # -1 and (price_buy - price.Close_Bid[i])/price_buy>=_filtre_multipoz:
        
            _pnl = (price.Close_Ask.iloc[i] - _price_buy_mean) * _size * _open_buy
            _open_buy += 1
            _equity = _cash + _pnl
            EQUITY.append(_equity)
            CASH.append(_cash)
            _position = 1
            #_index_entry = 0
            _nbtransactions += 1
            price_buy = price.Close_Ask.iloc[i]
            PRICE_BUY.append(price_buy)
            _price_buy_mean = round(sum(PRICE_BUY)/len(PRICE_BUY),5)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print('Position',_open_buy,'bought at ', price_buy,'(verification liste',PRICE_BUY[-1],')')
                print(col.Fore.YELLOW,'Nouvelle moyenne du price_buy',col.Style.RESET_ALL,_price_buy_mean)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(1)
            CLOSE_POZ.append(0)
            RATE_OPEN_POZ.append(price_buy)
            RATE_CLOSE_POZ.append(0)
            PNL_LAT.append(_pnl)
            PNL_REAL.append(0)
            TOTAL_OPEN.append(1) 
            TOTAL_PNL_LAT.append(_pnl)
            TOTAL_PNL_REAL.append(0) 


        elif (float(price.Low_Bid.iloc[i]) - float(_price_buy_mean))/float(_price_buy_mean) >= _target \
            or ((float(price.Low_Bid.iloc[i]) - float(_price_buy_mean) > 0) and (i - _index_entry > 200)):#slowd[i]>80:
    
            _position = 0
            _pnl = (price.Low_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
            _total += _pnl
            _cash += _pnl
            _equity = _cash
            EQUITY.append(_equity)
            CASH.append(_cash)
            _winner += _open_buy

            TRADE_DURATION.append(i - _index_entry)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                if _pnl > 0:
                    print(_open_buy,'positions TP closed at',price.Low_Bid.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                else:
                    print(_open_buy,'positions TP closed at',price.Low_Bid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(1)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(price.Low_Bid.iloc[i])
            PNL_LAT.append(0)
            PNL_REAL.append(_pnl)
            TOTAL_PNL_LAT.append(0)
            TOTAL_PNL_REAL.append(_pnl)
            TOTAL_CLOSE.append(_open_buy)
            PRICE_BUY = [] 
            _open_buy = 0

        elif ((float(price.Low_Bid.iloc[i]) - float(_price_buy_mean))/float(_price_buy_mean) <= -_sl) or \
            ((i - _index_entry) > 50 and (float(price.Low_Bid.iloc[i]) - float(_price_buy_mean)) < 0):
            _position = 0
            _pnl = (price.High_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
            _total += _pnl
            _cash += _pnl
            _equity = _cash
            EQUITY.append(_equity)
            CASH.append(_cash)
            _looser += _open_buy

            TRADE_DURATION.append(i - _index_entry)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                if _pnl < 0:
                    print(_open_buy,'positions SL closed at',price.High_Bid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                else :
                    print(_open_buy,'positions SL closed at',price.High_Bid.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(1)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(price.High_Bid.iloc[i])
            PNL_LAT.append(0)
            PNL_REAL.append(_pnl)
            TOTAL_CLOSE.append(_open_buy) 
            TOTAL_PNL_LAT.append(0)
            TOTAL_PNL_REAL.append(_pnl)
            PRICE_BUY = []
            _open_buy = 0
        else:

            _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
            _equity = _cash + _pnl
            EQUITY.append(_equity)
            CASH.append(_cash)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(0)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(0) ############### 
            PNL_LAT.append(_pnl)
            PNL_REAL.append(0)
            TOTAL_PNL_LAT.append(_pnl)
            TOTAL_PNL_REAL.append(0) 


    elif _position == -1:
        
        if price.Signal[i] == 1 : # 1 and (price.Close_Bid[i] - price_sell)/price_sell>=_filtre_multipoz :
            
            _pnl = - (price.Low_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
            _open_sell += 1
            _equity = _cash + _pnl
            EQUITY.append(_equity)
            CASH.append(_cash)
            _position = -1
            #_index_entry = 0
            _nbtransactions += 1
            price_sell = price.Close_Bid.iloc[i]
            PRICE_SELL.append(price_sell)
            _price_sell_mean = round(sum(PRICE_SELL)/len(PRICE_SELL),5)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print('Position 1 sold at', price_sell,'(verification liste',PRICE_SELL[-1],')')
                print(col.Fore.YELLOW,'Nouvelle moyenne du price_sell',col.Style.RESET_ALL,_price_sell_mean)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(1)
            CLOSE_POZ.append(0)
            RATE_OPEN_POZ.append(price_sell)
            RATE_CLOSE_POZ.append(0)
            PNL_LAT.append(_pnl)
            PNL_REAL.append(0)
            TOTAL_OPEN.append(1) 
            TOTAL_PNL_LAT.append(_pnl)
            TOTAL_PNL_REAL.append(0) 



        elif (float(price.High_Ask.iloc[i]) - float(_price_sell_mean))/float(_price_sell_mean) <= -_target \
            or ((float(price.High_Ask.iloc[i]) - float(_price_sell_mean) < 0) and (i - _index_entry > 200)):
            _position = 0
            _pnl = - (price.High_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
            _total += _pnl
            _cash += _pnl
            _equity = _cash
            EQUITY.append(_equity)
            CASH.append(_cash)
            _winner += _open_sell
            TRADE_DURATION.append(i - _index_entry)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                if _pnl > 0 :
                    print(_open_sell,'positions TP closed at',price.High_Ask.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                else:
                    print(_open_sell,'positions TP closed at',price.High_Ask.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(-1)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(price.High_Ask.iloc[i])
            PNL_LAT.append(0)
            PNL_REAL.append(_pnl)
            TOTAL_PNL_LAT.append(0)
            TOTAL_PNL_REAL.append(_pnl)
            TOTAL_CLOSE.append(_open_sell)
            PRICE_SELL = [] 
            _open_sell = 0

        elif ((float(price.Low_Ask.iloc[i]) - float(_price_sell_mean))/float(_price_sell_mean) >= _sl) or \
            ((i - _index_entry) > 50 and (float(price.Low_Ask.iloc[i]) - float(_price_sell_mean)>0)) :
            _position = 0
            _pnl = - (price.Low_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
            _total += _pnl
            _cash += _pnl
            _equity = _cash
            EQUITY.append(_equity)
            CASH.append(_cash)
            _looser += _open_sell
            TRADE_DURATION.append(i - _index_entry)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                if _pnl < 0 :    
                    print(_open_sell,'position SL closed at',price.Low_Ask.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                else:
                    print(_open_sell,'position SL closed at',price.Low_Ask.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(-1)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(price.Low_Ask.iloc[i])
            PNL_LAT.append(0)
            PNL_REAL.append(_pnl)
            TOTAL_PNL_LAT.append(0)
            TOTAL_PNL_REAL.append(_pnl)
            TOTAL_CLOSE.append(_open_sell)
            PRICE_SELL = []
            _open_sell = 0

        else:

            _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
            _equity = _cash + _pnl

            EQUITY.append(_equity)
            CASH.append(_cash)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(0)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(0)
            PNL_LAT.append(_pnl)
            PNL_REAL.append(0)
            TOTAL_PNL_LAT.append(_pnl)

            TOTAL_PNL_REAL.append(0)
            
    elif i == (len(price)-1) and _position != 0 :

        if _position == -1:
            _position = 0
            _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
            _total += _pnl
            _cash += _pnl
            _equity = _cash
            EQUITY.append(_equity)
            CASH.append(_cash)
            _looser += _open_sell
            TRADE_DURATION.append(i - _index_entry)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print(col.Fore.CYAN,"Cloture des positions en l'air",col.Style.RESET_ALL)
                print(_open_sell,'position closed at',price.Close_Ask.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(-1)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(price.Close_Ask.iloc[i])
            PNL_LAT.append(0)
            PNL_REAL.append(_pnl)
            TOTAL_PNL_LAT.append(0)
            TOTAL_PNL_REAL.append(_pnl)
            TOTAL_CLOSE.append(_open_sell)
            PRICE_SELL = []
            _open_sell = 0

        if _position == 1:

            _position = 0
            _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
            _total += _pnl
            _cash += _pnl
            _equity = _cash
            EQUITY.append(_equity)
            CASH.append(_cash)
            _looser += _open_buy

            TRADE_DURATION.append(i - _index_entry)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print(col.Fore.CYAN,"Cloture des positions en l'air",col.Style.RESET_ALL)
                print(_open_buy,'positions closed at',price.Close_Bid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(1)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(price.Close_Bid.iloc[i])
            PNL_LAT.append(0)
            PNL_REAL.append(_pnl)
            TOTAL_CLOSE.append(_open_buy) 
            TOTAL_PNL_LAT.append(0)
            TOTAL_PNL_REAL.append(_pnl)
            PRICE_BUY = []
            _open_buy = 0

try:
    _average_duration = round(statistics.mean(TRADE_DURATION),2)
    _max_duration = max(TRADE_DURATION)
    _min_duration = min([item for item in TRADE_DURATION if item !=0])

except:
    print("(pas de duration)") 
    _average_duration = 'NA' 
print(col.Fore.BLUE,'Pour le ticker',col.Fore.YELLOW,x,col.Style.RESET_ALL)
if _total > 0:              
    print(col.Fore.MAGENTA,"\nLes gains faramineux s'élèvent à : $",col.Fore.GREEN,round(_total,2),'!. En ',\
        _nbtransactions,col.Style.RESET_ALL,' transactions.' )
else:
    print(col.Fore.MAGENTA,"\nLes gains faramineux s'élèvent à : $",col.Fore.RED,round(_total,2),'!. En ',\
        _nbtransactions,col.Style.RESET_ALL,' transactions.' ) 
print(col.Fore.GREEN,"\nNbre de winners :",_winner,col.Style.RESET_ALL)
print(col.Fore.RED,"\nNbre de loosers :",_looser,col.Style.RESET_ALL)

backtest_graph['Equity'] = EQUITY

df_resultats[x] = [_period,_period2,(round(_equity,2)),(_winner),(_looser),(_max_duration),(_min_duration),(_average_duration),(_total)]

DER_POZ.append(_pnl)

engine.say("Finito caucau")
engine.runAndWait()




 Le rate du ticker EUR/USD est à  1 
Chargement des bases
Bases chargées
Préparation des dataframes
TETEL process effectué
 ENTERING THE BACKTEST 
 2019-01-03 00:00:00 
Position 1 sold at 1.1327 (verification liste 1.1327 )
 2019-01-03 00:05:00 
Position 1 sold at 1.13309 (verification liste 1.13309 )
 Nouvelle moyenne du price_sell  1.1329
 2019-01-03 00:10:00 
Position 1 sold at 1.13358 (verification liste 1.13358 )
 Nouvelle moyenne du price_sell  1.13312
 2019-01-03 00:15:00 
Position 1 sold at 1.13368 (verification liste 1.13368 )
 Nouvelle moyenne du price_sell  1.13326
 2019-01-03 00:20:00 
Position 1 sold at 1.13394 (verification liste 1.13394 )
 Nouvelle moyenne du price_sell  1.1334
 2019-01-03 00:25:00 
Position 1 sold at 1.1341 (verification liste 1.1341 )
 Nouvelle moyenne du price_sell  1.13352
 2019-01-03 00:30:00 
Position 1 sold at 1.13411 (verification liste 1.13411 )
 Nouvelle moyenne du price_sell  1.1336
 2019-01-03 00:35:00 
Position 1 sold at 1.13388 (verificatio

In [11]:
df_historical = pd.DataFrame()
df_historical = pd.DataFrame(index=DATE)
df_historical['Contract'] = CONTRACT
df_historical['Open_Poz'] = OPEN_POZ
df_historical['Close_Pos'] = CLOSE_POZ
df_historical['Rate_Open_Poz'] = RATE_OPEN_POZ 
df_historical['Rate_Close_Poze'] = RATE_CLOSE_POZ
df_historical['Pnl_Lat'] = TOTAL_PNL_LAT
df_historical['Pnl_Real'] = TOTAL_PNL_REAL
df_historical = df_historical.sort_index()
_generated_cash = round(df_historical.Pnl_Real.sum() * _rate,2)
_generated_cash_perc = round((_generated_cash / _cash_ini) * 100,2)
print(col.Fore.YELLOW,x,col.Fore.BLUE,'results',col.Style.RESET_ALL)
print(col.Fore.MAGENTA,'Tested Period',_year_bottom,' à',_year_top,col.Style.RESET_ALL)
print(col.Fore.CYAN,'Total Number of trades',max([sum(TOTAL_OPEN),sum(TOTAL_CLOSE)]),col.Style.RESET_ALL)
if _generated_cash <= 0:
    print('Started Cash :',_size)
    print('P&L in currency:',col.Fore.RED,str(_generated_cash)+'$',col.Style.RESET_ALL)
    print('P&L in %:',col.Fore.RED,str(_generated_cash_perc)+'%',col.Style.RESET_ALL)

else:
    print('Started Cash :',_size)
    print('P&L  in currency:',col.Fore.GREEN,str(_generated_cash)+'$',col.Style.RESET_ALL)
    print('P&L in %:',col.Fore.GREEN,str(_generated_cash_perc)+'%',col.Style.RESET_ALL)

#print('Pnl latent minimal :',str(round((df_historical.Pnl_Real.min()) * _rate,2))+'$')
#print('Pnl latent maximal :',str(round((df_historical.Pnl_Real.max())* _rate,2))+'$')
#print('Pnl latent moyen :',str(round(df_historical.Pnl_Real.mean()* _rate,2))+'$')
#print('Maximal trade duration',_max_duration)
#print('Minimal trade duration',_min_duration)
print('Average trade duration',_average_duration)
print('# Winners ',df_resultats.T['Nbre Winners'].sum())
print('# Loosers ',df_resultats.T['Nbre Loosers'].sum())
print('Cumulated gains',round(df_historical[df_historical.Pnl_Real>0].Pnl_Real.sum() * _rate,2))
print('Cumulated losses',round(df_historical[df_historical.Pnl_Real<0].Pnl_Real.sum() * _rate,2))
print(col.Fore.BLUE,'PROFIT FACTOR : ',\
    abs(round(df_historical[df_historical.Pnl_Real>0].Pnl_Real.sum()/df_historical[df_historical.Pnl_Real<0].Pnl_Real.sum(),2)),col.Style.RESET_ALL)
print(col.Fore.CYAN,'Winners Ratio :',\
    round((df_resultats.T['Nbre Winners'].sum()*100)/(df_resultats.T['Nbre Loosers'].sum()+df_resultats.T['Nbre Winners'].sum()),2),'%',col.Style.RESET_ALL)
print('Average Winners',round(sum(list(filter(lambda x:  x > 0,PNL_REAL)))/sum(TOTAL_OPEN) * _rate,2))
print('Average Loosers',round(sum(list(filter(lambda x:  x < 0,PNL_REAL)))/sum(TOTAL_OPEN) * _rate,2))
print('Average pnl',round(sum(PNL_REAL)/sum(TOTAL_OPEN) * _rate,2))
print('% Average Winners',round(sum(list(filter(lambda x:  x > 0,PNL_REAL)))/sum(TOTAL_OPEN) * _rate * 100 / _cash_ini,2))
print('% Average Loosers',round(sum(list(filter(lambda x:  x < 0,PNL_REAL)))/sum(TOTAL_OPEN) * _rate / _cash_ini * 100,2))
print('% Average pnl',round((sum(PNL_REAL)/len(set(PNL_REAL)) * _rate) / _cash_ini * 100,2))
print('Number of opened trades',sum(TOTAL_OPEN))
print('Number of closed trades',sum(TOTAL_CLOSE))
#print("Nombre de positions en l'air",(sum(TOTAL_OPEN) - sum(TOTAL_CLOSE)))

 EUR/USD  results 
 Tested Period 2019-01-01 00:00:01  à 2019-12-31 23:59:59 
 Total Number of trades 901 
Started Cash : 5000
P&L in currency:  -12324.45$ 
P&L in %:  -6.16% 
Average trade duration 232.25
# Winners  0
# Loosers  901
Cumulated gains 405.15
Cumulated losses -12729.6
 PROFIT FACTOR :  0.03 
 Winners Ratio : 0.0 % 
Average Winners 0.45
Average Loosers -14.13
Average pnl -13.68
% Average Winners 0.0
% Average Loosers -0.01
% Average pnl -1.23
Number of opened trades 901
Number of closed trades 901


In [ ]:
###############
### CONSECUTIVE PATTERN
###############

In [90]:
price2.head(1)

,Open_Bid,High_Bid,Low_Bid,Close_Bid,Open_Ask,High_Ask,Low_Ask,Close_Ask,Open,High,Low,Close,Range_Bid,Range_Ask,Range,OpenBid_Shift,OpenAsk_Shift,Open_Shift,Signal,TimeRange
Date,,,,,,,,,,,,,,,,,,,,
2020-01-01 22:00:00,1.11979,1.12387,1.11959,1.12217,1.11994,1.12399,1.11992,1.12234,1.119865,1.12393,1.119755,1.122255,NaN,NaN,NaN,1.12217,1.12234,1.122255,0,2020-01-01


In [11]:
_year_bottom = '2010-01-01 00:00:01'
_year_top = '2020-12-31 23:59:59'

data2 = pd.read_csv('Base_Clean/'+x.replace('/','')+'W1.csv')

data2['Date'] = data2['Date'].shift(-1)
data2.index = pd.to_datetime(data2['Date'])
data2 = data2.drop(['Date','Total'],axis=1)
data2.rename(columns={'OpenBid':'Open_Bid','HighBid':'High_Bid','LowBid':'Low_Bid','CloseBid':'Close_Bid',\
    'OpenAsk':'Open_Ask','HighAsk':'High_Ask','LowAsk':'Low_Ask','CloseAsk':'Close_Ask'},inplace = True)

data2 = data2[(data2.index >= _year_bottom) & (data2.index <= _year_top)]

price2 = data2.copy()

if '/JPY' in x:
    _pip_factor = 100
else:
    _pip_factor = 10000
price2['Range_Bid'] = (price2.Open_Bid.shift(1) - price2.Close_Bid.shift(1)) * _pip_factor
price2['Range_Ask'] = (price2.Open_Ask.shift(1) - price2.Close_Ask.shift(1)) * _pip_factor
price2['Range'] = (price2.Open.shift(1) - price2.Close.shift(1)) * _pip_factor

price2['Signal'] = np.where((price2.Range_Ask >= 80) | (price2.Range_Bid >= 80) | (price2.Range >= 80),1,\
    np.where((price2.Range_Ask <= -80) | (price2.Range_Bid <= -80) | (price2.Range <= -80),-1,0))
price2['TimeRange']= [price2.index[i].strftime(format='%Y-%m-%d') for i in range(len(price2))]

pattern_data=[0,-1,-1,0]
data=list(price2.Signal)

count=0
for i in range(0,len(data),1):
    if data[i:i+len(pattern_data)]==pattern_data:
        print(i,data[i:i+len(pattern_data)])
        count+=1

print(count)

4 [0, -1, -1, 0]
40 [0, -1, -1, 0]
91 [0, -1, -1, 0]
126 [0, -1, -1, 0]
192 [0, -1, -1, 0]
209 [0, -1, -1, 0]
357 [0, -1, -1, 0]
384 [0, -1, -1, 0]
494 [0, -1, -1, 0]
530 [0, -1, -1, 0]
729 [0, -1, -1, 0]
1302 [0, -1, -1, 0]
1354 [0, -1, -1, 0]
1562 [0, -1, -1, 0]
1598 [0, -1, -1, 0]
2163 [0, -1, -1, 0]
16


In [ ]:
#############
####### CLEAN BASE FOR WEEKLY
#############

In [8]:
import datetime as dt
import colorama as col
import pandas as pd
import multiprocessing.dummy as mp 
import os
import time
import configparser

config = configparser.ConfigParser()

config.read('config.ini')

#_period1 = m5
#_period2 = H1
#_period3 = D1
#_period4 = W1
#_period5 = M1
#_period6 = m15
#_period7 = m30
#_period8 = H4

_period8 = config.get('TIMEFRAME','_period8') 
_path1 = config.get('PATH','_path1') # 'Base/'

TICKERS = config.get('TICKERS','TICKERS') # TICKERS
TICKERS = TICKERS.split(',')



def scrap_ini(x):
   
    try:
        globals()['df1_%s' %x.replace('/','')] = pd.read_csv(_path1+x.replace('/','')+'_'+_period+'_BidAndAsk.csv',delimiter='|')
        print('\r',col.Fore.BLUE,'Reindexation de la base',col.Fore.YELLOW,x,col.Style.RESET_ALL,end='',flush=True)
    
        globals()['df1_%s' %x.replace('/','')] = \
            globals()['df1_%s' %x.replace('/','')].set_index(pd.to_datetime(globals()['df1_%s' %x.replace('/','')].Date,format='%Y-%m-%d %H:%M:%S'),drop=True)
        globals()['df1_%s' %x.replace('/','')] = globals()['df1_%s' %x.replace('/','')].drop(['Date'],axis=1)
        globals()['df1_%s' %x.replace('/','')].columns = ['OpenBid', 'HighBid','LowBid','CloseBid','OpenAsk','HighAsk','LowAsk','CloseAsk','Total','None']
        globals()['df1_%s' %x.replace('/','')] = globals()['df1_%s' %x.replace('/','')].drop(['None'],axis=1)
        globals()['df1_%s' %x.replace('/','')]['OpenBid'] = \
        globals()['df1_%s' %x.replace('/','')]['OpenBid'].replace(to_replace=',',value='.',regex=True).astype(float)
        globals()['df1_%s' %x.replace('/','')]['HighBid'] = \
        globals()['df1_%s' %x.replace('/','')]['HighBid'].replace(to_replace=',',value='.',regex=True).astype(float)
        globals()['df1_%s' %x.replace('/','')]['LowBid'] = \
        globals()['df1_%s' %x.replace('/','')]['LowBid'].replace(to_replace=',',value='.',regex=True).astype(float)
        globals()['df1_%s' %x.replace('/','')]['CloseBid'] = \
        globals()['df1_%s' %x.replace('/','')]['CloseBid'].replace(to_replace=',',value='.',regex=True).astype(float)
        globals()['df1_%s' %x.replace('/','')]['OpenAsk'] = \
        globals()['df1_%s' %x.replace('/','')]['OpenAsk'].replace(to_replace=',',value='.',regex=True).astype(float)
        globals()['df1_%s' %x.replace('/','')]['HighAsk'] = \
        globals()['df1_%s' %x.replace('/','')]['HighAsk'].replace(to_replace=',',value='.',regex=True).astype(float)
        globals()['df1_%s' %x.replace('/','')]['LowAsk'] = \
        globals()['df1_%s' %x.replace('/','')]['LowAsk'].replace(to_replace=',',value='.',regex=True).astype(float)
        globals()['df1_%s' %x.replace('/','')]['CloseAsk'] = \
        globals()['df1_%s' %x.replace('/','')]['CloseAsk'].replace(to_replace=',',value='.',regex=True).astype(float)

        globals()['df1_%s' %x.replace('/','')]['Open'] = (globals()['df1_%s' %x.replace('/','')].OpenBid + globals()['df1_%s' %x.replace('/','')].OpenAsk)/2
        globals()['df1_%s' %x.replace('/','')]['High'] = (globals()['df1_%s' %x.replace('/','')].HighBid + globals()['df1_%s' %x.replace('/','')].HighAsk)/2
        globals()['df1_%s' %x.replace('/','')]['Low'] = (globals()['df1_%s' %x.replace('/','')].LowBid + globals()['df1_%s' %x.replace('/','')].LowAsk)/2
        globals()['df1_%s' %x.replace('/','')]['Close'] = (globals()['df1_%s' %x.replace('/','')].CloseBid + globals()['df1_%s' %x.replace('/','')].CloseAsk)/2
        if _period == 'm5':
            globals()['df1_%s' %x.replace('/','')]['HigMax'] = \
                globals()['df1_%s' %x.replace('/','')]['HighBid'].iloc[::-1].rolling(99).max().iloc[::-1]
            globals()['df1_%s' %x.replace('/','')]['LowMin'] = \
                globals()['df1_%s' %x.replace('/','')]['LowAsk'].iloc[::-1].rolling(99).min().iloc[::-1]


        globals()['df1_%s' %x.replace('/','')].to_csv(_path2+x.replace('/','')+_period+'.csv')
    except:
        print('Problème pour le ticker',x,'en timeframe',_period)
    return()

def clean_base():
    global _period
    _t1 = dt.datetime.now()
    _period = _period8
    print('Début des opérations pour la timeframe',_period)
    p=mp.Pool(os.cpu_count())
    p.map(scrap_ini,TICKERS) 
    p.close()
    p.join()
    print("\n\n ===> Opération terminée. Tout est nettoyé et prêt à l'utilisation pour la time frame",_period)
    _t2 = dt.datetime.now()
    print("Temps d'excution du module",str((_t2 - _t1)))
    return()

if __name__ == "__main__":
    pass

##### NETTOYAGE DES BASES POUR LES 3 TIMEFRAMEs (prévoir 16 min)

import configparser
from scraptickers import scrap_tickers

config = configparser.ConfigParser()

config.read('config.ini')
_period1 = config.get('TIMEFRAME','_period1') # 'm5'
_period2 = config.get('TIMEFRAME','_period2') # 'H1'
_period3 = config.get('TIMEFRAME','_period3') # 'D1'
_period4 = config.get('TIMEFRAME','_period4') # 'W1'
_path1 = config.get('PATH','_path1') # 'Base/'
_path2 = config.get('PATH','_path2') # 'Base_Clean/'
_path3 = config.get('PATH','_path3') # 'Base_Input/'
TICKERS = scrap_tickers(_path1)
config['TICKERS']['TICKERS'] = \
            str(TICKERS).replace("'",'').replace('[','').replace(']','').replace(',','').replace(' ',',')

config['PATH']['_path'] = _path1

with open('config.ini', 'w') as configfile:    # save
    config.write(configfile)


clean_base()

Base/
Début des opérations pour la timeframe H4
  Reindexation de la base  GBP/NZD Problème pour le ticker XLM/USD en timeframe H4
Problème pour le ticker BCH/USD en timeframe H4
Problème pour le ticker BTC/USD en timeframe H4
  Reindexation de la base  CAD/JPY 

 ===> Opération terminée. Tout est nettoyé et prêt à l'utilisation pour la time frame H4
Temps d'excution du module 0:00:23.746385


()

In [14]:
from sklearn.linear_model import LinearRegression
import colorama as col
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import seaborn as sns
import time
import os
import datetime as dt
import numpy as np
import fxcmpy
import socketio
import shelve
import talib
import statistics
import pyttsx3
import datetime
import pytz
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
import matplotlib.ticker as mpticker
import pandas
import numpy
import mplfinance as mpf
from mpl_finance import candlestick_ohlc

In [38]:
%%time

x = 'EUR/USD'

_rate = 1

_year_bottom = '2020-01-01 00:00:01'
_year_top = '2020-12-31 23:59:59'

print(col.Fore.MAGENTA,'Le rate du ticker',x,'est à ',_rate,col.Style.RESET_ALL)

print('Chargement des bases')

data = pd.read_csv('Base_Clean/'+x.replace('/','')+'m5.csv',parse_dates=True)
data2 = pd.read_csv('Base_Clean/'+x.replace('/','')+'D1.csv',parse_dates=True)

data.index = pd.to_datetime(data['Date'])
data = data.drop(['Date','Total'],axis=1)
data.rename(columns={'OpenBid':'Open_Bid','HighBid':'High_Bid','LowBid':'Low_Bid','CloseBid':'Close_Bid',\
    'OpenAsk':'Open_Ask','HighAsk':'High_Ask','LowAsk':'Low_Ask','CloseAsk':'Close_Ask'},inplace = True)

#data2['Date'] = data2['Date'].shift(-1)
data2.index = pd.to_datetime(data2['Date'])
data2 = data2.drop(['Date','Total'],axis=1)
data2.rename(columns={'OpenBid':'Open_Bid','HighBid':'High_Bid','LowBid':'Low_Bid','CloseBid':'Close_Bid',\
    'OpenAsk':'Open_Ask','HighAsk':'High_Ask','LowAsk':'Low_Ask','CloseAsk':'Close_Ask'},inplace = True)

data = data[(data.index >= _year_bottom) & (data.index <= _year_top)]
data2 = data2[(data2.index >= _year_bottom) & (data2.index <= _year_top)]

price = data.copy()
price2 = data2.copy()

print('Bases chargées')
print('Préparation des dataframes')

#if '/JPY' in x:
#    _pip_factor = 100
#else:
#    _pip_factor = 10000
#price2['Range_Bid'] = (price2.Open_Bid.shift(1) - price2.Close_Bid.shift(1)) * _pip_factor
#price2['Range_Ask'] = (price2.Open_Ask.shift(1) - price2.Close_Ask.shift(1)) * _pip_factor
#price2['Range'] = (price2.Open.shift(1) - price2.Close.shift(1)) * _pip_factor
#price2['OpenBid_Shift'] = price2['Open_Bid'].shift(-1)
#price2['OpenAsk_Shift'] = price2['Open_Ask'].shift(-1)
#price2['Open_Shift'] = price2['Open'].shift(-1)
#price2['Signal'] = np.where((price2.Range_Ask >= _atr) | (price2.Range_Bid >= _atr) | (price2.Range >= _atr),1,\
#    np.where((price2.Range_Ask <= -_atr) | (price2.Range_Bid <= -_atr) | (price2.Range <= -_atr),-1,0))
#price2['TimeRange']= [price2.index[i].isocalendar()[1] for i in range(len(price2))]
price['TimeRange'] = [(price.index[i]-dt.timedelta(days=price.index[i].weekday()+3)).strftime(format='%Y-%m-%d') for i in range(len(price))]
#price2['TimeRange'] = price2['TimeRange']+1
price.TimeRange = price.TimeRange.shift(-12)




 Le rate du ticker EUR/USD est à  1 
Chargement des bases
Bases chargées
Préparation des dataframes
CPU times: user 3.36 s, sys: 236 ms, total: 3.59 s
Wall time: 3.6 s


In [45]:
%%time



def get_optimum_clusters(df, saturation_point=0.05):
    '''
    :param df: dataframe
    :param saturation_point: The amount of difference we are willing to detect
    :return: clusters with optimum K centers
    This method uses elbow method to find the optimum number of K clusters
    We initialize different K-means with 1..10 centers and compare the inertias
    If the difference is no more than saturation_point, we choose that as K and move on
    '''
    
    wcss = []
    k_models = []

    size = min(11, len(df.index))
    for i in range(1, size):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=500, n_init=10, random_state=0)
        kmeans.fit(df)
        wcss.append(kmeans.inertia_)
        k_models.append(kmeans)

    # Compare differences in inertias until it's no more than saturation_point
    optimum_k = len(wcss)-1
    for i in range(0, len(wcss)-1):
        diff = abs(wcss[i+1] - wcss[i])
        if diff < saturation_point:
            optimum_k = i
            break

    print("Optimum K is " + str(optimum_k + 1))
    optimum_clusters = k_models[optimum_k]

    return (optimum_clusters)



def mydate(x,pos):
    try:
        return datetime.datetime.fromtimestamp(x, tz=est).strftime(date_format)
    except IndexError:
        return ''


symbol = 'EUR/USD'

data = data2.copy() #30 1M candles
data = data.iloc[-30:]

data["Time"] = [d.timestamp() for d in pd.to_datetime(data.index.strftime('%Y-%m-%d'))]

data = data[["Time", "Open", "High", "Low", "Close"]]

lod, hod = 0, 0

lows = pandas.DataFrame(data=data, index=data.index, columns=["Low"])
highs = pandas.DataFrame(data=data, index=data.index, columns=["High"])

low_clusters = get_optimum_clusters(lows)
low_centers = low_clusters.cluster_centers_
low_centers = numpy.sort(low_centers, axis=0)

high_clusters = get_optimum_clusters(highs)
high_centers = high_clusters.cluster_centers_
high_centers = numpy.sort(high_centers, axis=0)

#print(data)
print(low_centers)
print(high_centers)

mpf.plot(data,
            hlines=dict(hlines=[low_centers[i][0] for i in range(len(low_centers))]+[high_centers[i][0] for i in range(len(high_centers))],
            colors=['orange','blue'],
            linestyle='-.'),
            type='candle',
            title='EUR/USD Chépatrokan',
            ylabel = 'Q.I. des huîtres')
            
            #figscale=1.4,savefig='test-mplfiance.png')#,
            #style='charles',
            #volume=False, 
            #mav=(3,6,9)), 
            #savefig='test-mplfiance.png'))

plt.show()

Optimum K is 1
Optimum K is 1
[[1.17984083]]
[[1.18746183]]
CPU times: user 940 ms, sys: 36.7 ms, total: 977 ms
Wall time: 771 ms


In [40]:
data.index

DatetimeIndex(['2020-08-03 21:00:00', '2020-08-04 21:00:00',
               '2020-08-05 21:00:00', '2020-08-06 21:00:00',
               '2020-08-08 21:00:00', '2020-08-09 21:00:00',
               '2020-08-10 21:00:00', '2020-08-11 21:00:00',
               '2020-08-12 21:00:00', '2020-08-13 21:00:00',
               '2020-08-15 21:00:00', '2020-08-16 21:00:00',
               '2020-08-17 21:00:00', '2020-08-18 21:00:00',
               '2020-08-19 21:00:00', '2020-08-20 21:00:00',
               '2020-08-22 21:00:00', '2020-08-23 21:00:00',
               '2020-08-24 21:00:00', '2020-08-25 21:00:00',
               '2020-08-26 21:00:00', '2020-08-27 21:00:00',
               '2020-08-29 21:00:00', '2020-08-30 21:00:00',
               '2020-08-31 21:00:00', '2020-09-01 21:00:00',
               '2020-09-02 21:00:00', '2020-09-03 21:00:00',
               '2020-09-05 21:00:00', '2020-09-06 21:00:00'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [33]:
[d.timestamp() for d in pd.to_datetime(data.index.strftime('%Y-%m-%d'))]

[1596412800.0,
 1596499200.0,
 1596585600.0,
 1596672000.0,
 1596844800.0,
 1596931200.0,
 1597017600.0,
 1597104000.0,
 1597190400.0,
 1597276800.0,
 1597449600.0,
 1597536000.0,
 1597622400.0,
 1597708800.0,
 1597795200.0,
 1597881600.0,
 1598054400.0,
 1598140800.0,
 1598227200.0,
 1598313600.0,
 1598400000.0,
 1598486400.0,
 1598659200.0,
 1598745600.0,
 1598832000.0,
 1598918400.0,
 1599004800.0,
 1599091200.0,
 1599264000.0,
 1599350400.0]

In [28]:
price2

,Open_Bid,High_Bid,Low_Bid,Close_Bid,Open_Ask,High_Ask,Low_Ask,Close_Ask,Open,High,Low,Close
Date,,,,,,,,,,,,
2020-01-01 22:00:00,1.12217,1.12217,1.11629,1.11714,1.12234,1.12234,1.11641,1.11729,1.122255,1.122255,1.116350,1.117215
2020-01-02 22:00:00,1.11714,1.11793,1.11244,1.11554,1.11729,1.11805,1.11255,1.11615,1.117215,1.117990,1.112495,1.115845
2020-01-04 22:00:00,1.11554,1.11651,1.11554,1.11650,1.11615,1.11719,1.11615,1.11719,1.115845,1.116850,1.115845,1.116845
2020-01-05 22:00:00,1.11650,1.12049,1.11566,1.11964,1.11719,1.12061,1.11578,1.11980,1.116845,1.120550,1.115720,1.119720
2020-01-06 22:00:00,1.11964,1.11970,1.11329,1.11518,1.11980,1.11982,1.11342,1.11551,1.119720,1.119760,1.113355,1.115345
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-01 21:00:00,1.19107,1.19282,1.18217,1.18541,1.19126,1.19294,1.18226,1.18562,1.191165,1.192880,1.182215,1.185515
2020-09-02 21:00:00,1.18541,1.18640,1.17884,1.18508,1.18562,1.18653,1.17896,1.18524,1.185515,1.186465,1.178900,1.185160
2020-09-03 21:00:00,1.18508,1.18647,1.17802,1.18350,1.18524,1.18660,1.17814,1.18412,1.185160,1.186535,1.178080,1.183810
